# STEP 1: User specifies the protein family and the taxonomic group. Using the relevant EMBOSS functionalities to 1) obtain the relevant protein sequence data



In [1]:
#!/usr/bin/python3
import os
import subprocess
import sys
import re
import numpy as np
import pandas as pd

In [6]:
# the following script takes an input from the user, asking them which database they'd like to search from
# and then asks them which search item they'd like to search for and what search type this search item is.
# then it asks whether they'd like to search for a partial match or not.

# TODO: Intro to the programme
# Welcome the user to the programme, introduce what the programme does and what input it will take from the user
print("Welcome to the E-Search programme!"
      "\nThis programme is used for searching protein sequences within the protein database"
      "\nIt will start by asking the user to specify their search terms, and this can be either of the following:"
      "\n1) The taxonomic group name, "
      "\n2) A refined search term or "
      "\n3) The UID of the protein of interest")



Welcome to the E-Search programme!
This programme is used for searching protein sequences within the protein database
It will start by asking the user to specify their search terms, and this can be either of the following:
1) The taxonomic group name, 
2) A refined search term or 
3) The UID of the protein of interest


# Get user_input and do quality check for the search term
## Note that esearch doesn't work within jupyter notebook

In [31]:
### PROCESS STEP 0: INPUT AND INPUT PRE-PROCESSING ###
## This step takes the user's input and store it as a variable so that it can be fed into the esearch commandline

# define a function to get confirmation from the user as to whether they'd like to proceed (This function is universal to be used in different situations)
def get_confirmation():
    '''This function gets confirmation from the user as to whether they'd like to proceed. It returns True if the user would indeed like to proceed'''
    while True:
        # take an input from the user and turn it lowercase
        confirmation = input("Would you like to proceed? (y/n)").lower()
        if confirmation == 'y':
            return True
        elif confirmation == 'n':
            return False
        else:
            print("Invalid input. Please enter 'y' or 'n'")

# define a function to check the quality of the user's input
def quality_check_user_input(user_input):
    '''This function checks the quality of the user's input. It checks whether the user has specified a valid taxonomic group or not.
    If the user has specified a valid taxonomic group, then the function returns True, otherwise it returns False.'''
    # screen for invalid inputs
    # if the user_input is empty, then the user has not specified a taxonomic group and they need to specify the input again
    if re.search("^$",user_input):  # using re, search for anything that contains nothing betweent the start and end of the string
        return False
    # screen for invalid search term
    try:
        # run esearch in the taxonomy database and save it to esearch_user_input
        esearch_user_input = subprocess.getoutput("esearch -db taxonomy -spell -query"+'"' + user_input +'"'+ "| efetch")
    except subprocess.CalledProcessError as e:
        print(f"Error executing subprocess: {e}, please run the programme again and provide an appropriate taxonomic group name.")
        # handle the error or exit the programme
        sys.exit(1)
    
    # if esearch returns an empty line on NCBI, return False
    if re.search("^$", esearch_user_input):
        return False
    
    # if esearch returns a warning or error on NCBI, return False, else True
    if "FAILURE" in esearch_user_input or "WARNING" in esearch_user_input or "ERROR" in esearch_user_input:
        print("Invalid input. Please enter 'y' or 'n'") 
        return False
    else:
        return True

In [6]:
#TODO: define a function to refine search result 
def refine_search_term():
    '''This function collects a further user input to refine our search result. Input quality check is also performed.'''
    refined_input = input("How would you like to refine your search term? Enter its name to proceed. ")
    refined_input_type = input("What is the search term type?")
    # while loop to make sure user_input passes the quality check, this will run until it passes!
    while True:
        if quality_check_user_input(user_input):
            print("The following search term will be used in NCBI: ", user_input, '\n',
                  '*Note: This may not be the exact name of the taxonomic group as NCBI allows room for typo in the search term.*')
            return user_input
        else:
            print("The taxonomic group you have specified is not valid. Please try again.")
            # ask user to specify the taxonomic group they'd like to search for AGAIN
            user_input = input("Which taxonomic group would you like to search for? Enter its name to proceed. ")
            continue

# define a function to perform a search within the NCBI protein database
def protein_esearch(search_term):
    '''This function performs an esearch within the NCBI protein database given a valid input and outputs the fasta sequence of the protein specified.'''
    user_result = subprocess.getoutput("esearch -db protein -spell -query " + search_term + "| efetch -format fasta")
    # This will print all the fasta sequences found to the screen
    print(user_result)
    #TODO: if the user wants to save the fasta sequence as a file
    
# define a function to get the scientific names of the taxonomic groups
def get_scientific_names():
    '''The function collects the scientific names from user_input. Quality check is also performed'''
    # search for this taxonomic group on esearch and save the output in user_result
    user_result = subprocess.getoutput("esearch -db taxonomy -spell -query " + user_input + "| efetch")
    # search for this taxonomic group on esearch and save the names of the possible taxonimic groups to a list
    print("")
    scientific_names = subprocess.getoutput("esearch -db taxonomy -spell -query " + user_input + "| efetch -format docsum | xtract -pattern DocumentSummary -element ScientificName")  # -spell performs spell check, 'efetch -format docsum' gets the DocSum of the query, and 'xtract -pattern DocumentSummary -element ScientificName' extracts the scientific name 
    # saving every item within scientific_names into a list
    scientific_name_list = scientific_names.split("\n")  # splitting the list into a list of scientific names
    
    
    #TODO: Maybe put if statements here?
    #TODO: Return the more refined input.
    print("The search term ", user_input, " returns the following results from NCBI:",
          user_result,".\nIts scientific name is: ", scientific_name_list)
    return scientific_name_list, scientific_names, user_result

In [ ]:
# define a function to get the user's input
def user_input_UID_False():
    '''This function gets user_input if the user is not inputting an UID as a search item'''
    user_input = input("Which taxonomic group would you like to search for? Enter its name to proceed.")
    # check the quality of the user's input
    while True:
        if quality_check_user_input(user_input) == True:
            print("The taxonomic group you have specified is valid.")
            print(f"The taxonomic group you have specified is: {user_input}")
            print("The programme will now proceed to search for the specified taxonomic group.")
            print("Please wait...")
            return user_input
        # if the has not specified a valid taxonomic group then they need to specify the input again
        else:
            print("The taxonomic group you have specified is not valid. Please try again.")
            # ask user to specify the taxonomic group they'd like to search for AGAIN
            user_input = input("Which taxonomic group would you like to search for? Enter its name to proceed: ")
            continue
 


def get_input():
    '''This function collects the user input.'''
    # ask the user if they'd like to search for a UID or start from a taxonomy group
    while True:
        # ask the user if the search term is a UID or a defined search term
        user_search_type = input("Is your search term a UID? (y/n)").lower()
        # if the search term entered by the user is a TAXONOMIC GROUP i.e. not UID
        if user_search_type == 'n':
            # pass through the function to check the quality of the user's input
            user_input = user_input_UID_False()
            return True

        # if the search term entered by the user is a refined search term or a UID
        elif user_search_type == 'y':
            # pass through the function to check the quality of the user's input
            user_input = input("Please enter a UID: ")
            protein_esearch(user_input)

            # TODO

            return False
        # if the search type is not properly defined
        else:
            print("Invalid input. Please enter 'y' or 'n'")

# save user_input as a global variable, but also allow the user to interrupt the programme
try:
    user_input = get_input()
except KeyboardInterrupt:
    print("\nProgramme interrupted by the user.")
    sys.exit(0)

In [1]:
# execute get_scientific_names to save the outputs as global variables
scientific_name_list, scientific_names, user_result = get_scientific_names()

#TODO: quality check scientific_name_list (low priority)
# if the scientific_name_list is empty, then the user has not specified a valid taxonomic group and they need to specify the input again
# (this should not happen, but a sanity check is always good!)
if len(scientific_name_list) == 0:
    # use the get_input function
    user_input = get_input()
    # use the get_scientific_names function
    scientific_name_list, scientific_names, user_result = get_scientific_names()
    print("here")
    # once the condition for scientific_name_list != 0 is satisfied, continue to the next if statement
    exit()
    
# if there is only one scientific_name in scientific_name_list, then the user has specified 1 valid taxonomic group
if len(scientific_name_list) == 1:
    print("You have specified the taxonomic group: ", scientific_names,
          "\nLet's refine your results further...")
    user_confirmation = get_confirmation()
    print(user_confirmation) # debug line
    # if the user would like to proceed, then we can further refine the search
    if user_confirmation == True:
        ######### 

    # if the user would like to change their taxonomic group name, then they have to specify the taxonomic group again
    if user_confirmation == False:
        scientific_name = input("Which taxonomic group would you like to search for? ")
        scientific_names = subprocess.getoutput("esearch -db taxonomy -spell -query "+user_input+ "| efetch -format docsum | xtract -pattern DocumentSummary -element ScientificName")  # -spell performs spell check
        # splitting scientific_names into a list of scientific name
        scientific_name_list = scientific_names.split("\n")  # splitting the list into a list of scientific names
        #TODO: This will then take them to the beginning of this if statement


if len(scientific_name_list) > 1:
    


# output to the screen asking the user which taxonomic group they'd like to search for
input("Which of the following taxonomic group would you like to search for?","\n",scientific_name_list)

IndentationError: expected an indented block (2807924904.py, line 39)

In [ ]:
#TODO: THIS SHOULD BE RAN IN BIOINF SERVER
# specify the database to search from, the user cannot specify the database they'd like to search from, or this will become too complicated...
database = "protein"
print("The search database is set to ", database)

# ask user which search type they'd like to use
SearchType1 = input("Which search type would you like to use? ")
# ask user which search item they'd like to search for
SearchItem2 = input("Which search item would you like to search for? ")

# ask user which search type they'd like to use
SearchType2 = input("Which search type would you like to use? ")

# ask user whether they'd like to search for a partial match or not
partial_or_not = input("Would you like to search for a partial match or not? ")

# confirm with the user what they've entered by printing the input features to the screen
print("You have entered: \n"
      "Search database:", str(database))

use subprocess to run the following esearch
subprocess.call("esearch -db {database} -query "{SearchItem1}[{SearchType1}] AND {SearchItem2}[{SearchType2}] {partial_or_not}", shell=True) # should ask user whether they'd like it partial or not, default should be NOT PARTIAL
for example: esearch -db nucleotide -query "Cosmoscarta[organism]"
use of efetch for the sequence data:
esearch -db protein -query "Homo sapiens" | efetch -format fasta >Homosapiens.fasta

In [5]:
    #!/usr/bin/python3
import os
import subprocess
import sys
import re
import numpy as np
import pandas as pd

sequences =  subprocess.getoutput("cat "+"AvesORGN_AND_glucose6phosphatasePROT"+".fasta").split('>')[1:]
new_file_name = "AvesORGN_AND_glucose6phosphatasePROT"
# define a function to extract the sequences from the fasta file to a folder containing all sequences as fasta files
def extract_seq(file_name):
    '''This function takes the fasta sequence containing multiple sequences as an input and extracts the sequences to separate files.
    The separate files are saved in a new folder and 
    '''

    # create a new directory to save the extracted sequences
    new_dir = str(str(file_name) + "_sequences")
    os.mkdir(f"{new_dir}")
    # split the sequences within the fasta file into individual sequences
    # get the content of the fasta file into a variable and splitting the content into individual sequences
    sequences = subprocess.getoutput("cat "+str(file_name)+".fasta").split('>')[1:]
    # define an empty dictionary to collect seq_data
    seq_data_dict = {}
    # extract the sequence data of each sequence in the fasta file
    for sequence in sequences:
        # split the sequence into individual
        lines = sequence.split('\n')
        # save the accession ID by taking the first line in lines and splitting it by space
        header = lines[0].split(' ')
        # the first item in header is the accession ID
        accession_id = header[0]
        # join all the lines together to get the whole sequence
        seq_data = ''.join(lines[1:])
        # use a dictionary to save each item in result_name_dict as a value and its header as a key
        seq_data_dict[accession_id] = seq_data
        # save the sequence data to a new file
        with open(f"./{new_dir}/{accession_id}" + ".fasta", "w") as f:
            f.write(">"+str(header)+"\n")
            f.write(seq_data)
            f.close()
    # inform the user where the files are saved
    print('The extracted sequences are saved in a new folder called', f'{new_dir}', '.')
    
    # return the dictionary of sequence data and the header
    return seq_data_dict

seq_data_dict = extract_seq(new_file_name)
print(seq_data_dict)

The extracted sequences are saved in a new folder called AvesORGN_AND_glucose6phosphatasePROT_sequences .
{}


In [9]:
import pandas as pd
df = pd.read_csv("motif_counts.csv")
df

,Unnamed: 0,ATP_GTP_A,ABC_TRANSPORTER_1,RGD,LEUCINE_ZIPPER,MICROBODIES_CTER,AMIDATION
0,AAA91199.1.fasta\n,1,1.0,0.0,0.0,0.0,0.0
1,AAB36585.1.fasta\n,1,1.0,1.0,2.0,0.0,0.0
2,AAB36586.1.fasta\n,1,1.0,1.0,2.0,0.0,0.0
3,AAB36587.1.fasta\n,1,1.0,1.0,2.0,0.0,0.0
4,AAB36588.1.fasta\n,1,1.0,1.0,2.0,0.0,0.0
5,AAF76889.1.fasta\n,1,1.0,0.0,0.0,1.0,0.0
6,AAP44087.1.fasta\n,1,0.0,0.0,0.0,0.0,0.0
7,BAB13728.2.fasta\n,1,1.0,0.0,1.0,0.0,1.0
8,BAB71769.1.fasta\n,1,1.0,1.0,2.0,0.0,0.0
9,CAA05793.1.fasta\n,1,1.0,0.0,0.0,0.0,2.0


In [1]:
num_subplots = 6
# Calculate the number of rows and columns based on the number of subplots
num_rows = (num_subplots // 2) + (num_subplots % 2)
num_cols = 2

# Adjust the figsize based on the number of subplots
figsize = (12, num_rows * 5)  # You can adjust the multiplier as needed
print(figsize)

(12, 15)
